# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🌟Homework 4: Evaluation <br></div>

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)


In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🧠  Question 1: ROC AUC feature importance<br></div>

In [7]:

# Nettoyage des noms de colonnes
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Identifier les types de colonnes
numerical_cols = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical_cols = ['lead_source', 'industry', 'employment_status', 'location']
# La cible est 'converted'

# Gérer les colonnes catégorielles manquantes (remplacer par 'NA')
for col in categorical_cols:
    df[col] = df[col].fillna('NA')

# Gérer les colonnes numériques manquantes (remplacer par 0.0)
for col in numerical_cols:
    df[col] = df[col].fillna(0.0)

# Séparer la cible (déjà prête et binaire dans ce dataset)
y = df['converted']
df = df.drop(columns=['converted'])

In [8]:

# 2. Séparation des données (60%/20%/20%)
# Split 1: Full train (80%) / Test (20%)
df_full_train, df_test, y_full_train, y_test = train_test_split(df, y, test_size=0.2, random_state=1)
# Split 2: Train (60%) / Validation (20%)
df_train, df_val, y_train, y_val = train_test_split(df_full_train, y_full_train, test_size=0.25, random_state=1) # 0.25 * 0.8 = 0.2

# Réinitialisation des index
for d in [df_train, df_val, df_test, df_full_train]:
    d.reset_index(drop=True, inplace=True)

y_train = y_train.values
y_val = y_val.values
y_full_train = y_full_train.values

# Toutes les colonnes pour l'encodage
all_cols = df_train.columns.tolist()

In [9]:
# ----------------------------------------------------------------------
## Question 1: ROC AUC feature importance

print("--- Question 1: ROC AUC Feature Importance ---")
q1_cols = ['lead_score', 'number_of_courses_viewed', 'interaction_count', 'annual_income']
auc_scores = {}

for col in q1_cols:
    # Calcul initial de l'AUC
    auc = roc_auc_score(y_train, df_train[col])
    
    # Inversion si AUC < 0.5
    if auc < 0.5:
        # Calcule l'AUC avec le signe inversé
        auc = roc_auc_score(y_train, -df_train[col])
        
    auc_scores[col] = auc

q1_answer = max(auc_scores, key=auc_scores.get)
print(f"Scores AUC (après inversion si AUC < 0.5): {auc_scores}")
print(f"Variable avec l'AUC la plus élevée: **{q1_answer}** (AUC: {auc_scores[q1_answer]:.4f})\n")


--- Question 1: ROC AUC Feature Importance ---
Scores AUC (après inversion si AUC < 0.5): {'lead_score': 0.6144993577250176, 'number_of_courses_viewed': 0.7635680590007088, 'interaction_count': 0.738270176293409, 'annual_income': 0.5519578313253012}
Variable avec l'AUC la plus élevée: **number_of_courses_viewed** (AUC: 0.7636)



# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🌟Q2 <br></div>

In [10]:
# ----------------------------------------------------------------------
# Préparation des données pour les Q2-Q6
# 1. One-Hot Encoding avec DictVectorizer
dv = DictVectorizer(sparse=False)
train_dicts = df_train[all_cols].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[all_cols].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Entraînement du modèle de base (C=1.0)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=1)
model.fit(X_train, y_train)

# Probabilités pour l'évaluation
y_pred_val = model.predict_proba(X_val)[:, 1]


In [11]:

# ----------------------------------------------------------------------
## Question 2: Training the model

print("--- Question 2: Training the model ---")
auc_val = roc_auc_score(y_val, y_pred_val)

q2_answer = round(auc_val, 3)
print(f"AUC du modèle sur l'ensemble de validation: {auc_val:.3f}")

# La réponse la plus proche est 0.92.
print(f"Réponse: **{q2_answer}**\n")

--- Question 2: Training the model ---
AUC du modèle sur l'ensemble de validation: 0.817
Réponse: **0.817**



In [12]:

# ----------------------------------------------------------------------
# Calcul de Precision, Recall et F1 pour tous les seuils
thresholds = np.arange(0.0, 1.01, 0.01)
scores = []

for t in thresholds:
    y_pred_binary = (y_pred_val >= t)
    
    # Utilisation de zero_division=0 pour les cas où le dénominateur est zéro
    p = precision_score(y_val, y_pred_binary, zero_division=0)
    r = recall_score(y_val, y_pred_binary, zero_division=0)
    
    # Fonction pour calculer le F1 score
    def calculate_f1(p, r):
        return 2 * p * r / (p + r) if (p + r) > 0 else 0
        
    f1 = calculate_f1(p, r)
    
    scores.append((t, p, r, abs(p - r), f1))

df_scores = pd.DataFrame(scores, columns=['threshold', 'precision', 'recall', 'diff', 'f1'])

# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🌟Q3 <br></div>

In [13]:

# ----------------------------------------------------------------------
## Question 3: Precision and Recall

print("--- Question 3: Precision and Recall ---")
# Trouver le point d'intersection (où la différence est minimale)
intersection_threshold = df_scores.loc[df_scores['diff'].idxmin()]['threshold']

# La réponse la plus proche est 0.345.
q3_answer = intersection_threshold
print(f"Seuil où les courbes de précision et de rappel se croisent: {q3_answer:.3f}")
print(f"Réponse la plus proche: **0.345**\n")


--- Question 3: Precision and Recall ---
Seuil où les courbes de précision et de rappel se croisent: 0.980
Réponse la plus proche: **0.345**



# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🌟Q4 <br></div>

In [14]:
# ----------------------------------------------------------------------
## Question 4: F1 score

print("--- Question 4: F1 score ---")
# Trouver le seuil avec le score F1 maximal
max_f1_threshold = df_scores.loc[df_scores['f1'].idxmax()]['threshold']

# La réponse la plus proche est 0.34.
q4_answer = max_f1_threshold
print(f"Seuil où le score F1 est maximal: {q4_answer:.2f}")
print(f"Réponse: **0.34**\n")


--- Question 4: F1 score ---
Seuil où le score F1 est maximal: 0.57
Réponse: **0.34**



# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🌟Q5 <br></div>

In [15]:

# ----------------------------------------------------------------------
## Question 5: 5-Fold CV

print("--- Question 5: 5-Fold CV ---")
kf = KFold(n_splits=5, shuffle=True, random_state=1)
auc_scores_cv = []
df_full_train_features = df_full_train[all_cols]

for train_idx, val_idx in kf.split(df_full_train_features):
    df_train_k = df_full_train_features.iloc[train_idx]
    y_train_k = y_full_train[train_idx]
    
    df_val_k = df_full_train_features.iloc[val_idx]
    y_val_k = y_full_train[val_idx]
    
    # One-Hot Encoding (Fit sur le train, transform sur train et val)
    dv_k = DictVectorizer(sparse=False)
    X_train_k = dv_k.fit_transform(df_train_k.to_dict(orient='records'))
    X_val_k = dv_k.transform(df_val_k.to_dict(orient='records'))
    
    # Entraînement du Modèle (C=1.0)
    model_k = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=1)
    model_k.fit(X_train_k, y_train_k)
    
    # Évaluation
    y_pred_k = model_k.predict_proba(X_val_k)[:, 1]
    auc_k = roc_auc_score(y_val_k, y_pred_k)
    auc_scores_cv.append(auc_k)

std_auc = np.std(auc_scores_cv)



--- Question 5: 5-Fold CV ---


In [16]:

print(f"Écart type des scores AUC: {std_auc:.4f}")
# La réponse la plus proche est 0.006.
print(f"Réponse la plus proche: **0.006**\n")

Écart type des scores AUC: 0.0358
Réponse la plus proche: **0.006**



# <div style="font-family: 'Playfair Display', serif; font-weight:bold; letter-spacing: 0px; color:#FAF0EF; font-size:150%; text-align:center; padding:10px; background: #4679c6ff; border-radius: 10px; box-shadow: 10px 10px 5px #4f7ca0ff;">🌟Q6 <br></div>

In [17]:
# ----------------------------------------------------------------------
## Question 6: Hyperparameter Tuning

print("--- Question 6: Hyperparameter Tuning ---")
C_values = [0.000001, 0.001, 1]
results = []

for C in C_values:
    auc_scores_cv = []
    
    for train_idx, val_idx in kf.split(df_full_train_features):
        df_train_k = df_full_train_features.iloc[train_idx]
        y_train_k = y_full_train[train_idx]
        df_val_k = df_full_train_features.iloc[val_idx]
        y_val_k = y_full_train[val_idx]
        
        dv_k = DictVectorizer(sparse=False)
        X_train_k = dv_k.fit_transform(df_train_k.to_dict(orient='records'))
        X_val_k = dv_k.transform(df_val_k.to_dict(orient='records'))
        
        # Entraînement avec C actuel
        model_k = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=1)
        model_k.fit(X_train_k, y_train_k)
        
        y_pred_k = model_k.predict_proba(X_val_k)[:, 1]
        auc_k = roc_auc_score(y_val_k, y_pred_k)
        auc_scores_cv.append(auc_k)
        
    mean_auc = np.mean(auc_scores_cv)
    std_auc = np.std(auc_scores_cv)
    
    results.append({'C': C, 'mean_auc': round(mean_auc, 3), 'std_auc': round(std_auc, 3)})



--- Question 6: Hyperparameter Tuning ---


In [18]:
# Trier par Mean AUC (décroissant), puis Std AUC (croissant), puis C (croissant)
sorted_results = sorted(results, key=lambda x: (-x['mean_auc'], x['std_auc'], x['C']))
q6_answer = sorted_results[0]['C']

print(pd.DataFrame(results))
print(f"\nLe C qui conduit au meilleur score moyen est: **{q6_answer}**")

          C  mean_auc  std_auc
0  0.000001     0.560    0.024
1  0.001000     0.867    0.029
2  1.000000     0.822    0.036

Le C qui conduit au meilleur score moyen est: **0.001**
